# Using `ruff` in JupyterLab with `jupyterlab_code_formatter`

- **ruff** formatter: https://docs.astral.sh/ruff/ 
- **jupyterlab_code_formatter** JupyterLab plugin: https://jupyterlab-code-formatter.readthedocs.io/


# Watch the walkthrough video for details: https://youtu.be/FXA6PJUabKQ 

## Install `jupyterlab_code_formatter` 

This snippet creates a conda (mamba) environment called `example`.

```bash
mamba create -n example python=3.11
mamba activate example
mamba install -c conda-forge black ruff isort jupyterlab_code_formatter jupyterlab
```

## Generate (or if it exists, update) your jupyter config

This command will generate (or if it already exists ask if you want to overwrite) a jupyter config for your system.

```bash
jupyter lab --generate-config
```

This command returns the path to the new config.

## Copy and paste this code snippet to the end of your jupyter config file:

Special thanks to [Ryan Tam](https://twitter.com/ryantam626) for providing not only the `jupyterlab_code_formatter` plugin itself, but the modification code snippet below to make the new features of ruff available.

Misc links: 

```python
from typing import List
from jupyterlab_code_formatter.formatters import BaseFormatter, handle_line_ending_and_magic, SERVER_FORMATTERS, logger
import subprocess

class RuffFormatFormatter(BaseFormatter):

    label = "Apply Ruff Format Formatter - Confirmed working for 0.1.3"
    
    def __init__(self) -> None:
        try:
            from ruff.__main__ import find_ruff_bin

            self.ruff_bin = find_ruff_bin()
        except (ImportError, FileNotFoundError):
            self.ruff_bin = "ruff"

    @property
    def importable(self) -> bool:
        return True

    @handle_line_ending_and_magic
    def format_code(self, code: str, notebook: bool, args: List[str] = [], **options) -> str:
        process = subprocess.run(
            [self.ruff_bin, "format", "-"],
            input=code,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            encoding="utf-8",
        )

        if process.stderr:
            logger.info(process.stderr)
            return code
        else:
            return process.stdout

        
SERVER_FORMATTERS["ruff_format"] = RuffFormatFormatter()
```

Remember to restart your jupyter instance. 

## Start jupyter

    Edit the settings 
        > click the JSON editor 
            > Jupyterlab Code Formatter
                > Edit the python default formatter to say "ruff_format"

The top few lines will look like this after you've made this edit: 

```JSON
{
    "preferences": {
        "default_formatter": {
            "python": "ruff_format",
```

And now if you click the button in the toolbar, it will format the whole notebook using `ruff`. 

# Bonus -- if you want to run the formatter on only a single cell with a keyboard shortcut

## Add this snippet to the Jupyter keyboard shortcuts to enable formatting a single cell

    Edit the settings 
        > click the JSON editor 
            > Keyboard Shortcuts
                > Edit the right half (User preferences)

copy and paste this snippet to bind `Ctrl K` to format the cell.

```json
        {
            "command": "jupyterlab_code_formatter:format",
            "keys": [
                "Ctrl K"
            ],
            "selector": ".jp-Notebook.jp-mod-editMode",
            "args": {}
        },
```


Note, you have to be in Edit mode in the cell for this shortcut to work. 